# 04: Fetch Abstracts for Referenced Studies

## Objective
Match our extracted references to PubMed records and fetch abstracts for LLM evaluation.

## Matching Strategy (CrossRef-enhanced)

This workflow uses a 3-phase approach for maximum match rate:

1. **Direct Extraction** - Extract DOI/PMID directly from reference text using regex
2. **CrossRef API** - For references without DOI, query CrossRef's bibliographic search
3. **PubMed Lookup** - Use DOIs to fetch PMIDs, then batch-fetch abstracts

## Why CrossRef?
- CrossRef API (`query.bibliographic`) is specifically designed to match reference strings to DOIs
- Handles fuzzy matching internally (typos, ligatures, formatting issues)
- Has 130M+ DOIs - broader coverage than PubMed-only search
- DOI → PMID lookup is highly reliable (~95%+ when the paper exists in PubMed)

## Output
- `Data/referenced_paper_abstracts.csv` - Abstracts with match confidence scores

In [49]:
%pip install -q biopython pandas tqdm requests

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import re
import time
import os
import requests
from pathlib import Path
from tqdm.notebook import tqdm
from Bio import Entrez
from collections import Counter

# =============================================================================
# Configuration
# =============================================================================

# NCBI API configuration - with API key allows 10 requests/sec
# Credentials loaded from .env file (never hardcode!)
Entrez.email = os.environ.get("NCBI_EMAIL", "")
Entrez.api_key = os.environ.get("NCBI_API_KEY", "")

# Rate limits
NCBI_RATE = 0.11 if Entrez.api_key else 0.34  # seconds between NCBI requests
CROSSREF_RATE = 0.5  # seconds between CrossRef requests (polite)

# Setup paths
notebook_dir = Path.cwd()
project_root = notebook_dir if (notebook_dir / "Data").exists() else notebook_dir.parent
DATA_DIR = project_root / "Data"

# Input
REFS_CSV = DATA_DIR / "categorized_references.csv"
META_CSV = DATA_DIR / "review_metadata.csv"

# Output
OUTPUT_CSV = DATA_DIR / "referenced_paper_abstracts.csv"
PROGRESS_CSV = DATA_DIR / "crossref_matching_progress.csv"

print(f"Data directory: {DATA_DIR}")
print(f"NCBI API key configured: {bool(Entrez.api_key)}")
print(f"NCBI rate: {1/NCBI_RATE:.0f} requests/sec")
print(f"CrossRef rate: {1/CROSSREF_RATE:.0f} requests/sec")

Data directory: c:\Users\juanx\Documents\LSE-UKHSA Project\Data
NCBI API key configured: True
NCBI rate: 9 requests/sec
CrossRef rate: 2 requests/sec


In [51]:
# =============================================================================
# Load and Filter to Public Health Reviews
# =============================================================================

# Load all references
refs_df = pd.read_csv(REFS_CSV)
print(f"Total references: {len(refs_df):,}")

# Load metadata for filtering
meta_df = pd.read_csv(META_CSV)

# Define core public health Cochrane groups
PUBLIC_HEALTH_GROUPS = [
    'Public Health',
    'Tobacco Addiction', 
    'Drugs and Alcohol',
    'Infectious Diseases',
    'Acute Respiratory Infections',
    'HIV/AIDS',
    'STI',
]

# Filter to PH reviews
ph_reviews = meta_df[meta_df['cochrane_group'].isin(PUBLIC_HEALTH_GROUPS)]
ph_review_dois = set(ph_reviews['doi'].dropna())

print(f"\nPublic health reviews: {len(ph_reviews):,}")
print("Groups:", ", ".join([f"{g} ({(ph_reviews['cochrane_group']==g).sum()})" 
                            for g in PUBLIC_HEALTH_GROUPS if (ph_reviews['cochrane_group']==g).sum() > 0]))

# Filter references
refs_df = refs_df[refs_df['review_doi'].isin(ph_review_dois)].copy()
print(f"\nReferences after PH filter: {len(refs_df):,}")

Total references: 629,561

Public health reviews: 1,349
Groups: Public Health (79), Tobacco Addiction (231), Drugs and Alcohol (188), Infectious Diseases (361), Acute Respiratory Infections (463), STI (27)

References after PH filter: 66,608


In [52]:
# =============================================================================
# Deduplicate References
# =============================================================================

def create_ref_signature(row):
    """Create a normalized signature for deduplication."""
    title = str(row.get('title', '')).lower().strip()[:100]
    year = str(row.get('year', ''))
    first_author = str(row.get('authors', '')).split()[0].lower() if row.get('authors') else ''
    return f"{first_author}|{year}|{title[:50]}"

refs_df['signature'] = refs_df.apply(create_ref_signature, axis=1)
unique_refs = refs_df.drop_duplicates(subset='signature').copy()

print(f"Total references: {len(refs_df):,}")
print(f"Unique references: {len(unique_refs):,}")
print(f"Deduplication ratio: {len(refs_df)/len(unique_refs):.1f}x")
print(f"\nCategories:")
print(unique_refs['category'].value_counts().to_string())

Total references: 66,608
Unique references: 41,840
Deduplication ratio: 1.6x

Categories:
category
excluded    26885
included    13025
awaiting     1217
ongoing       713


In [53]:
# =============================================================================
# PHASE 1: Extract DOI/PMID directly from reference text
# =============================================================================
# Many references already have DOI or PMID embedded in the text

def extract_pmid(text):
    """Extract PMID from reference text."""
    if pd.isna(text):
        return None
    match = re.search(r"(PMID|PUBMED|MEDLINE)[:\s]*(\d+)", str(text), flags=re.I)
    if match:
        return match.group(2)
    return None

def extract_doi(text):
    """Extract DOI from reference text."""
    if pd.isna(text):
        return None
    match = re.search(
        r"(?:DOI[:\s]*|HTTPS?://(?:DX\.)?DOI\.ORG/)?(10\.\d{4,9}/[-._;()/:A-Z0-9]+)", 
        str(text), flags=re.I
    )
    if match:
        doi = match.group(1)
        return doi.rstrip(".,;)")  # remove trailing punctuation
    return None

# Build full reference text for extraction
unique_refs['full_ref'] = (
    unique_refs['title'].fillna('') + ' ' + 
    unique_refs['authors'].fillna('') + ' ' +
    unique_refs['year'].fillna('').astype(str)
)

# Extract DOI and PMID
unique_refs['extracted_doi'] = unique_refs['full_ref'].apply(extract_doi)
unique_refs['extracted_pmid'] = unique_refs['full_ref'].apply(extract_pmid)

# Also use ref_doi and pmid columns if available
unique_refs['final_doi'] = unique_refs['extracted_doi'].combine_first(unique_refs.get('ref_doi'))
unique_refs['final_pmid'] = unique_refs['extracted_pmid'].combine_first(unique_refs.get('pmid'))

has_doi = unique_refs['final_doi'].notna().sum()
has_pmid = unique_refs['final_pmid'].notna().sum()

print("PHASE 1: Direct Extraction")
print("=" * 60)
print(f"References with DOI: {has_doi:,} ({has_doi/len(unique_refs)*100:.1f}%)")
print(f"References with PMID: {has_pmid:,} ({has_pmid/len(unique_refs)*100:.1f}%)")
print(f"Need CrossRef lookup: {len(unique_refs) - has_doi:,}")

PHASE 1: Direct Extraction
References with DOI: 2,746 (6.6%)
References with PMID: 993 (2.4%)
Need CrossRef lookup: 39,094


C:\Users\juanx\AppData\Local\Temp\ipykernel_81992\1551778175.py:41: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  unique_refs['final_pmid'] = unique_refs['extracted_pmid'].combine_first(unique_refs.get('pmid'))


In [ ]:
# =============================================================================
# CrossRef and PubMed API Functions
# =============================================================================

def clean_reference(ref):
    """Clean reference text for CrossRef query."""
    if pd.isna(ref):
        return ""
    ref = str(ref)
    # Remove Cochrane-specific text
    ref = re.sub(r"\(REVIEW\).*", "", ref, flags=re.I)
    ref = re.sub(r"TRUSTED EVIDENCE.*", "", ref, flags=re.I)
    ref = re.sub(r"COCHRANE.*?LIBRARY", "", ref, flags=re.I)
    # Remove page markers
    ref = re.sub(r"---\s*Page\s*\d+\s*---", " ", ref)
    # Remove excessive whitespace
    ref = re.sub(r"\s{2,}", " ", ref)
    return ref.strip()


def get_doi_from_crossref(ref, timeout=10):
    """Query CrossRef API to get DOI from bibliographic reference."""
    if not ref or len(ref) < 20:
        return None
    
    try:
        url = "https://api.crossref.org/works"
        params = {"query.bibliographic": ref, "rows": 1}
        headers = {"User-Agent": f"LSE-UKHSA-Project/1.0 (mailto:{Entrez.email})" if Entrez.email else "LSE-UKHSA-Project/1.0"}
        
        r = requests.get(url, params=params, headers=headers, timeout=timeout)
        r.raise_for_status()
        
        items = r.json().get("message", {}).get("items", [])
        if items:
            return items[0].get("DOI")
    except Exception as e:
        pass
    return None


def get_pmid_from_doi(doi, api_key=None):
    """Look up PMID from DOI via PubMed."""
    if not doi:
        return None
    
    try:
        params = {
            "db": "pubmed",
            "term": f"{doi}[DOI]",
            "retmode": "json"
        }
        if api_key:
            params["api_key"] = api_key
            
        r = requests.get(
            "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi",
            params=params, timeout=10
        )
        data = r.json()
        idlist = data.get("esearchresult", {}).get("idlist", [])
        return idlist[0] if idlist else None
    except Exception:
        return None


def fetch_abstracts_batch(pmids, batch_size=200, max_retries=3):
    """Batch fetch PubMed records (200 per request) with retry logic."""
    results = {}
    pmid_list = [str(int(p)) for p in pmids if pd.notna(p) and str(p).isdigit()]
    failed_batches = []
    
    for i in range(0, len(pmid_list), batch_size):
        batch = pmid_list[i:i + batch_size]
        success = False
        
        for attempt in range(max_retries):
            try:
                time.sleep(NCBI_RATE * (attempt + 1))  # Exponential backoff
                handle = Entrez.efetch(db="pubmed", id=",".join(batch), rettype="xml", retmode="xml")
                records = Entrez.read(handle)
                handle.close()
                
                for article in records.get('PubmedArticle', []):
                    data = extract_record_data(article)
                    if data:
                        results[data['pmid']] = data
                success = True
                break
            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"Batch {i//batch_size + 1} attempt {attempt + 1} failed: {e}, retrying...")
                else:
                    print(f"Batch {i//batch_size + 1} failed after {max_retries} attempts: {e}")
                    failed_batches.append(batch)
    
    if failed_batches:
        print(f"Warning: {len(failed_batches)} batch(es) failed permanently ({sum(len(b) for b in failed_batches)} PMIDs)")
    
    return results


def extract_record_data(record):
    """Extract fields from PubMed record."""
    try:
        article = record['MedlineCitation']['Article']
        pmid = str(record['MedlineCitation']['PMID'])
        title = str(article.get('ArticleTitle', ''))
        
        # Abstract
        abstract = ''
        if 'Abstract' in article and 'AbstractText' in article['Abstract']:
            parts = article['Abstract']['AbstractText']
            abstract = ' '.join([str(p) for p in parts]) if isinstance(parts, list) else str(parts)
        
        # Year
        year = ''
        if 'Journal' in article and 'JournalIssue' in article['Journal']:
            year = article['Journal']['JournalIssue'].get('PubDate', {}).get('Year', '')
        
        # Authors
        authors = []
        if 'AuthorList' in article:
            for auth in article['AuthorList']:
                if 'LastName' in auth:
                    name = auth['LastName'] + (' ' + auth.get('Initials', ''))
                    authors.append(name.strip())
        
        # DOI
        doi = ''
        if 'ELocationID' in article:
            for loc in article['ELocationID']:
                if loc.attributes.get('EIdType') == 'doi':
                    doi = str(loc)
                    break
        
        return {
            'pmid': pmid, 'title': title, 'abstract': abstract,
            'year': year, 'authors': '; '.join(authors), 'doi': doi
        }
    except:
        return None


print("API functions defined:")
print("  • get_doi_from_crossref() - CrossRef bibliographic search")
print("  • get_pmid_from_doi() - DOI to PMID lookup")
print("  • fetch_abstracts_batch() - Batch PubMed fetch")

API functions defined:
  • get_doi_from_crossref() - CrossRef bibliographic search
  • get_pmid_from_doi() - DOI to PMID lookup
  • fetch_abstracts_batch() - Batch PubMed fetch


In [55]:
# =============================================================================
# PHASE 2: CrossRef lookup for missing DOIs
# =============================================================================
# Query CrossRef API for references without DOI
# Progress saved every 500 refs

print("PHASE 2: CrossRef DOI Lookup")
print("=" * 60)

# References that need CrossRef lookup
refs_need_crossref = unique_refs[unique_refs['final_doi'].isna()].copy()
print(f"References needing CrossRef: {len(refs_need_crossref):,}")

# Time estimate
est_hours = len(refs_need_crossref) * CROSSREF_RATE / 3600
print(f"Estimated time: ~{est_hours:.1f} hours")

# Check for existing progress
if PROGRESS_CSV.exists():
    progress_df = pd.read_csv(PROGRESS_CSV)
    already_done = set(progress_df['study_id'])
    print(f"Resuming from checkpoint: {len(already_done):,} already processed")
else:
    progress_df = pd.DataFrame()
    already_done = set()

# Filter to remaining refs
refs_to_process = refs_need_crossref[~refs_need_crossref['study_id'].isin(already_done)]
print(f"Remaining to process: {len(refs_to_process):,}")

PHASE 2: CrossRef DOI Lookup
References needing CrossRef: 39,094
Estimated time: ~5.4 hours
Resuming from checkpoint: 32,130 already processed
Remaining to process: 0


In [34]:
# =============================================================================
# PHASE 2 EXECUTION: CrossRef DOI Lookup
# =============================================================================

results = []
start_time = time.time()
matched_count = 0

for idx, (_, row) in enumerate(tqdm(refs_to_process.iterrows(), total=len(refs_to_process), desc="CrossRef")):
    # Build reference string for CrossRef query
    ref_text = clean_reference(f"{row['title']} {row['authors']} {row['year']}")
    
    # Query CrossRef
    crossref_doi = get_doi_from_crossref(ref_text)
    time.sleep(CROSSREF_RATE)
    
    # Record result
    results.append({
        'study_id': row['study_id'],
        'category': row['category'],
        'original_title': row['title'],
        'original_authors': row['authors'],
        'original_year': row['year'],
        'crossref_doi': crossref_doi,
        'match_method': 'crossref' if crossref_doi else 'no_match'
    })
    
    if crossref_doi:
        matched_count += 1
    
    # Save progress every 500 refs
    if (idx + 1) % 500 == 0:
        batch_df = pd.DataFrame(results)
        combined = pd.concat([progress_df, batch_df], ignore_index=True)
        combined.to_csv(PROGRESS_CSV, index=False)
        
        elapsed = time.time() - start_time
        rate = (idx + 1) / elapsed * 3600
        print(f"\n[{idx+1:,}/{len(refs_to_process):,}] Saved. Match rate: {matched_count/(idx+1)*100:.1f}% ({rate:.0f}/hr)")

# Final save
if results:
    batch_df = pd.DataFrame(results)
    combined = pd.concat([progress_df, batch_df], ignore_index=True)
    combined.to_csv(PROGRESS_CSV, index=False)

print(f"\n✓ CrossRef complete: {matched_count:,} DOIs found ({matched_count/len(refs_to_process)*100:.1f}%)")

CrossRef:   0%|          | 0/39094 [00:00<?, ?it/s]


[500/39,094] Saved. Match rate: 99.6% (2557/hr)

[1,000/39,094] Saved. Match rate: 99.8% (2742/hr)

[1,500/39,094] Saved. Match rate: 99.9% (2844/hr)

[2,000/39,094] Saved. Match rate: 99.8% (2881/hr)

[2,500/39,094] Saved. Match rate: 99.8% (2881/hr)

[3,000/39,094] Saved. Match rate: 99.8% (2931/hr)

[3,500/39,094] Saved. Match rate: 99.8% (2949/hr)

[4,000/39,094] Saved. Match rate: 99.8% (2942/hr)

[4,500/39,094] Saved. Match rate: 99.8% (2959/hr)

[5,000/39,094] Saved. Match rate: 99.8% (2952/hr)

[5,500/39,094] Saved. Match rate: 99.8% (2969/hr)

[6,000/39,094] Saved. Match rate: 99.8% (2982/hr)

[6,500/39,094] Saved. Match rate: 99.8% (2986/hr)

[7,000/39,094] Saved. Match rate: 99.8% (2996/hr)

[7,500/39,094] Saved. Match rate: 99.9% (3007/hr)

[8,000/39,094] Saved. Match rate: 99.9% (3017/hr)

[8,500/39,094] Saved. Match rate: 99.7% (2918/hr)

[9,000/39,094] Saved. Match rate: 99.7% (2898/hr)

[9,500/39,094] Saved. Match rate: 99.7% (2867/hr)

[10,000/39,094] Saved. Match rat

In [57]:
# =============================================================================
# PHASE 3: Convert DOIs to PMIDs
# =============================================================================

print("PHASE 3: DOI → PMID Conversion")
print("=" * 60)

# Reload unique_refs fresh (to avoid column conflicts from previous runs)
unique_refs_fresh = refs_df.drop_duplicates(subset='signature').copy()

# Re-extract DOIs (Phase 1 logic)
unique_refs_fresh['full_ref'] = (
    unique_refs_fresh['title'].fillna('') + ' ' + 
    unique_refs_fresh['authors'].fillna('') + ' ' +
    unique_refs_fresh['year'].fillna('').astype(str)
)
unique_refs_fresh['extracted_doi'] = unique_refs_fresh['full_ref'].apply(extract_doi)
unique_refs_fresh['final_doi'] = unique_refs_fresh['extracted_doi']

# Combine with CrossRef DOIs
crossref_results = pd.read_csv(PROGRESS_CSV) if PROGRESS_CSV.exists() else pd.DataFrame()

if len(crossref_results) > 0:
    # Merge on normalized title (not study_id!) to correctly match different papers
    crossref_results['title_normalized'] = crossref_results['original_title'].str.lower().str.strip()
    unique_refs_fresh['title_normalized'] = unique_refs_fresh['title'].str.lower().str.strip()
    
    crossref_doi_map = crossref_results[['title_normalized', 'crossref_doi']].drop_duplicates()
    unique_refs_fresh = unique_refs_fresh.merge(
        crossref_doi_map,
        on='title_normalized',
        how='left'
    )
    unique_refs_fresh['final_doi'] = unique_refs_fresh['final_doi'].combine_first(unique_refs_fresh['crossref_doi'])

# Update the main variable
unique_refs = unique_refs_fresh

# Get all unique DOIs
all_dois = unique_refs[unique_refs['final_doi'].notna()]['final_doi'].str.lower().unique()
print(f"Total unique DOIs: {len(all_dois):,}")

# DOI→PMID cache file
DOI_PMID_CACHE = DATA_DIR / "doi_pmid_cache.csv"

# Load cached mappings if available (includes NO_PMID entries for DOIs without PubMed records)
doi_to_pmid = {}
no_pmid_cached = 0
if DOI_PMID_CACHE.exists():
    cache_df = pd.read_csv(DOI_PMID_CACHE)
    for _, row in cache_df.iterrows():
        doi = str(row['doi']).lower()
        pmid = str(row['pmid'])
        if pmid != 'NO_PMID':
            doi_to_pmid[doi] = pmid
        else:
            doi_to_pmid[doi] = None  # Mark as "looked up, no result"
            no_pmid_cached += 1
    print(f"Loaded {len(doi_to_pmid) - no_pmid_cached:,} cached DOI→PMID mappings")
    print(f"Loaded {no_pmid_cached:,} cached NO_PMID entries (not in PubMed)")

# Find DOIs not in cache at all
dois_to_lookup = [d for d in all_dois if d.lower() not in doi_to_pmid]
print(f"DOIs needing lookup: {len(dois_to_lookup):,}")

# Convert DOIs to PMIDs (only for uncached)
if dois_to_lookup:
    print("Converting DOIs to PMIDs...")
    new_mappings = 0
    for doi in tqdm(dois_to_lookup, desc="DOI→PMID"):
        pmid = get_pmid_from_doi(doi, Entrez.api_key)
        if pmid:
            doi_to_pmid[doi.lower()] = pmid
            new_mappings += 1
        else:
            doi_to_pmid[doi.lower()] = None  # Cache the "no result" too
        time.sleep(NCBI_RATE)
    
    # Save updated cache (convert None to 'NO_PMID' for storage)
    cache_rows = [{'doi': k, 'pmid': v if v else 'NO_PMID'} for k, v in doi_to_pmid.items()]
    cache_df = pd.DataFrame(cache_rows)
    cache_df.to_csv(DOI_PMID_CACHE, index=False)
    print(f"\\n✓ Found {new_mappings:,} new PMIDs, cache updated")

# Count actual PMIDs (not None)
actual_pmids = sum(1 for v in doi_to_pmid.values() if v is not None)
print(f"\\n✓ Total PMIDs available: {actual_pmids:,} ({actual_pmids/len(all_dois)*100:.1f}%)")

PHASE 3: DOI → PMID Conversion
Total unique DOIs: 31,927
Loaded 23,302 cached DOI→PMID mappings
Loaded 8,608 cached NO_PMID entries (not in PubMed)
DOIs needing lookup: 17
Converting DOIs to PMIDs...


DOI→PMID:   0%|          | 0/17 [00:00<?, ?it/s]

\n✓ Found 0 new PMIDs, cache updated
\n✓ Total PMIDs available: 23,302 (73.0%)


In [64]:
# =============================================================================
# PHASE 4: Batch Fetch Abstracts
# =============================================================================

print("PHASE 4: Fetch Abstracts")
print("=" * 60)

# Map DOIs to PMIDs
unique_refs['doi_lower'] = unique_refs['final_doi'].str.lower()
unique_refs['pmid_from_doi'] = unique_refs['doi_lower'].map(doi_to_pmid)

# Initialize final_pmid if it doesn't exist (fresh dataframe)
if 'final_pmid' not in unique_refs.columns:
    unique_refs['final_pmid'] = None
unique_refs['final_pmid'] = unique_refs['final_pmid'].combine_first(unique_refs['pmid_from_doi'])

# Get all unique PMIDs
all_pmids = unique_refs[unique_refs['final_pmid'].notna()]['final_pmid'].unique()
print(f"Total unique PMIDs: {len(all_pmids):,}")

# Batch fetch abstracts
print("Fetching abstracts in batches...")
start = time.time()
abstract_records = fetch_abstracts_batch(all_pmids, batch_size=200)
elapsed = time.time() - start

print(f"\n✓ Fetched {len(abstract_records):,} records in {elapsed:.0f}s")
with_abstract = sum(1 for r in abstract_records.values() if r.get('abstract'))
print(f"  With abstracts: {with_abstract:,} ({with_abstract/len(abstract_records)*100:.1f}%)")

PHASE 4: Fetch Abstracts
Total unique PMIDs: 23,182
Fetching abstracts in batches...

✓ Fetched 23,173 records in 602s
  With abstracts: 20,942 (90.4%)


In [65]:
# =============================================================================
# Compile Final Results
# =============================================================================
# Re-expand abstract data to ALL (study_id, review_doi) pairs via signature

print("Compiling final results...")
print("=" * 60)

# Build abstract data keyed by signature (from unique_refs)
signature_to_abstract = {}

for _, row in unique_refs.iterrows():
    sig = row['signature']
    pmid = str(row['final_pmid']) if pd.notna(row['final_pmid']) else None
    record = abstract_records.get(pmid, {}) if pmid else {}
    
    # Determine match method
    if pd.notna(row.get('extracted_pmid')):
        method = 'pmid_direct'
    elif pd.notna(row.get('extracted_doi')) or pd.notna(row.get('ref_doi')):
        method = 'doi_direct'
    elif pd.notna(row.get('crossref_doi')):
        method = 'crossref'
    else:
        method = 'no_match'
    
    signature_to_abstract[sig] = {
        'pmid': pmid,
        'doi': row['final_doi'],
        'matched_title': record.get('title', ''),
        'matched_authors': record.get('authors', ''),
        'matched_year': record.get('year', ''),
        'abstract': record.get('abstract', ''),
        'match_method': method if pmid else 'no_match'
    }

print(f"Abstract data built for {len(signature_to_abstract):,} unique signatures")

# Re-expand to ALL original (study_id, review_doi) pairs
output_rows = []

for _, row in refs_df.iterrows():
    sig = row['signature']
    abstract_data = signature_to_abstract.get(sig, {})
    
    output_rows.append({
        'study_id': row['study_id'],
        'review_doi': row['review_doi'],
        'category': row['category'],
        'original_title': row['title'],
        'original_authors': row['authors'],
        'original_year': row['year'],
        'pmid': abstract_data.get('pmid'),
        'doi': abstract_data.get('doi'),
        'matched_title': abstract_data.get('matched_title', ''),
        'matched_authors': abstract_data.get('matched_authors', ''),
        'matched_year': abstract_data.get('matched_year', ''),
        'abstract': abstract_data.get('abstract', ''),
        'match_method': abstract_data.get('match_method', 'no_match')
    })

results_df = pd.DataFrame(output_rows)

print(f"\nTotal references: {len(results_df):,}")
print(f"Unique (study_id, review_doi) pairs: {results_df.groupby(['study_id', 'review_doi']).ngroups:,}")
print(f"\nMatch methods:")
print(results_df['match_method'].value_counts().to_string())

Compiling final results...
Abstract data built for 41,840 unique signatures

Total references: 66,608
Unique (study_id, review_doi) pairs: 66,562

Match methods:
match_method
crossref      47121
no_match      19090
doi_direct      397


In [66]:
# =============================================================================
# Save Final Output (with review_doi preserved!)
# =============================================================================

# Filter to matched only
matched_refs = results_df[results_df['match_method'] != 'no_match'].copy()

print("FINAL RESULTS")
print("=" * 60)
print(f"Total matched: {len(matched_refs):,} / {len(results_df):,} ({len(matched_refs)/len(results_df)*100:.1f}%)")
print(f"Unique (study_id, review_doi) pairs: {matched_refs.groupby(['study_id', 'review_doi']).ngroups:,}")
print(f"\nBy category:")
print(matched_refs['category'].value_counts().to_string())

has_abstract = matched_refs['abstract'].str.len() > 0
print(f"\nWith abstracts: {has_abstract.sum():,} ({has_abstract.mean()*100:.1f}%)")

# Verify review_doi is in output
print(f"\n✓ Columns in output: {matched_refs.columns.tolist()}")

# Save
matched_refs.to_csv(OUTPUT_CSV, index=False)
print(f"\n✓ Saved to {OUTPUT_CSV}")

FINAL RESULTS
Total matched: 47,518 / 66,608 (71.3%)
Unique (study_id, review_doi) pairs: 47,483

By category:
category
excluded    30384
included    15921
awaiting      719
ongoing       494

With abstracts: 42,873 (90.2%)

✓ Columns in output: ['study_id', 'review_doi', 'category', 'original_title', 'original_authors', 'original_year', 'pmid', 'doi', 'matched_title', 'matched_authors', 'matched_year', 'abstract', 'match_method']

✓ Saved to c:\Users\juanx\Documents\LSE-UKHSA Project\Data\referenced_paper_abstracts.csv


In [67]:
# =============================================================================
# Summary Statistics
# =============================================================================

print("=" * 60)
print("PIPELINE SUMMARY")
print("=" * 60)

print(f"\nInput: {len(unique_refs):,} unique references from public health reviews")

print(f"\nPhase 1 - Direct extraction:")
direct_doi = unique_refs['extracted_doi'].notna().sum() if 'extracted_doi' in unique_refs.columns else 0
print(f"  DOIs extracted: {direct_doi:,}")

if PROGRESS_CSV.exists():
    crossref_df = pd.read_csv(PROGRESS_CSV)
    crossref_found = crossref_df['crossref_doi'].notna().sum()
    print(f"\nPhase 2 - CrossRef lookup:")
    print(f"  DOIs found: {crossref_found:,} / {len(crossref_df):,} ({crossref_found/len(crossref_df)*100:.1f}%)")

print(f"\nPhase 3 - DOI → PMID:")
actual_pmids = sum(1 for v in doi_to_pmid.values() if v is not None)
print(f"  PMIDs found: {actual_pmids:,}")

print(f"\nPhase 4 - Abstract fetch:")
print(f"  Records fetched: {len(abstract_records):,}")
print(f"  With abstracts: {sum(1 for r in abstract_records.values() if r.get('abstract')):,}")

print(f"\nFinal output: {OUTPUT_CSV.name}")
print(f"  Matched: {len(matched_refs):,} ({len(matched_refs)/len(unique_refs)*100:.1f}%)")
print(f"  By category: {dict(matched_refs['category'].value_counts())}")

PIPELINE SUMMARY

Input: 46,309 unique references from public health reviews

Phase 1 - Direct extraction:
  DOIs extracted: 279

Phase 2 - CrossRef lookup:
  DOIs found: 39,031 / 39,094 (99.8%)

Phase 3 - DOI → PMID:
  PMIDs found: 23,302

Phase 4 - Abstract fetch:
  Records fetched: 23,173
  With abstracts: 20,942

Final output: referenced_paper_abstracts.csv
  Matched: 47,518 (102.6%)
  By category: {'excluded': np.int64(30384), 'included': np.int64(15921), 'awaiting': np.int64(719), 'ongoing': np.int64(494)}
